In [1]:
# Добавляем текущую директорию в путь
import sys
import os

current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)
# проверка
from core.base_test import ModelReportBuilder, TestGroupBuilder
print("OK:")


OK:


In [5]:
# ================================================
# 0) Imports & Setup
# ================================================

import sys
import os

current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)
# проверка
%time from core.base_test import ModelReportBuilder, TestGroupBuilder
print("First check completed")

import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import KBinsDiscretizer

# === Ваши модули (импортируйте из своего пакета/пути) ===
# Импортируем базовые классы
from core.base_test import ModelReportBuilder, TestGroupBuilder

# Импортируем все доступные тесты M0 - качество данных
from tests.m0_data_quality import (
    M01_DataSummaryTest, M02_SampleOverlayTest, M03_ObsVsPredTest
)

# Импортируем все доступные тесты M1 - тесты распределения
from tests.m1_distribution_tests import (
    M11_TrainVsGenpopPSITest, M12_OOSVsGenpopPSITest, M13_MaxDateRecencyTest,
    M14_DataQualityOverviewTest, M15_MissingValuesTest
)

# Импортируем все доступные тесты M2 - Gini тесты  
from tests.m2_gini_tests import (
    M21_GiniTest, M22_GiniFactorsTest, M24_GiniDynamicsTest, M25_GiniUpliftTest
)

# Импортируем все доступные тесты M3 - мультиколлинеарность
from tests.m3_multicollinearity import (
    M33_MulticollinearityTest  # M31 помечен как NOT FOR USE
)

# Импортируем все тесты M4
from tests.m4_model_performance import (
    M41_MetricConfidenceIntervalTest, M42_SampleSizeAdequacyTest, M43_CalibrationTest,
    M44_OverfittingTest, M45_MetricStabilityTest, M46_CategoryQualityTest,
    M47_ShapImportanceTest, M48_FeatureContributionTest, M49_UpliftTest,
    M410_MissingValuesImpactTest, M411_TargetCorrelationTest, M412_FeatureCorrelationTest,
    M413_VIFTest, M414_TwoForestSelectionTest
)

# Импортируем все доступные тесты M5 - стабильность модели
from tests.m5_model_stability import (
    M51_ModelGiniStabilityTest, M52_FactorGiniStabilityTest, M53_ScorePSITest, M54_FactorPSITest
)

# Импортируем PSI калькулятор
from tests.psi_calculator import PSICalculator, PSICalculatorExtended
# Флаги тяжёлых тестов
RUN_SHAP = True     # M4.8 (и M4.7, если подключите)
RUN_UPLIFT = True    # M4.9 — относительно тяжёлый
print("All imports completed")


CPU times: total: 0 ns
Wall time: 0 ns
First check completed
All imports completed


In [6]:

# ================================================
# 1) Load California Housing & Build DataFrame
#    (делаем бинарную задачу классификации)
# ================================================
data = fetch_california_housing(as_frame=True)
df = data.frame.copy()
df.rename(columns={"MedHouseVal": "MedHouseVal"}, inplace=True)

# Колонки признаков (все числовые)
num_features = [
    "MedInc", "HouseAge", "AveRooms", "AveBedrms",
    "Population", "AveOccup", "Latitude", "Longitude"
]

# Бинаризуем таргет по медиане (классификация)
median_val = df["MedHouseVal"].median()
df["target"] = (df["MedHouseVal"] > median_val).astype(int)

# Категориальный признак для M4.6 (качество по категориям):
# бинируем Lat в 5 корзин
kb = KBinsDiscretizer(n_bins=5, encode="ordinal", strategy="quantile")
df["LatBin"] = pd.Series(kb.fit_transform(df[["Latitude"]]).astype(int).flatten(), index=df.index).astype("category")

# Дата: синтетическая месячная шкала (для M1.3, M4.5, M4.10)
rng = np.random.default_rng(42)
months = pd.date_range("2018-01-01", periods=60, freq="MS")
df["date"] = rng.choice(months, size=len(df))
df["date"] = pd.to_datetime(df["date"])

# Идентификатор
df["id"] = np.arange(len(df))

# Разбиваем на train/test 70/30 и проставляем sample
train_idx, test_idx = train_test_split(df.index, test_size=0.3, random_state=42, stratify=df["target"])
df["sample"] = "train"
df.loc[test_idx, "sample"] = "test"

# Обучаем быструю модель-классификатор и сохраняем pred/score
clf = RandomForestClassifier(n_estimators=500, random_state=42, n_jobs=-1)
clf.fit(df.loc[train_idx, num_features], df.loc[train_idx, "target"])
proba = clf.predict_proba(df[num_features])[:, 1]
pred = (proba >= 0.5).astype(int)

df["score"] = proba        # вероятности положительного класса (для classification)
df["pred"] = pred          # предсказанная метка класса

# Для M4.2/M4.9 понадобится класс модели (переобучения внутри тестов)
model_class_clf = RandomForestClassifier


In [7]:


# ================================================
# 2) Build unified CONFIG
# ================================================
config = {
    "task": "classification",  # мы сформировали бинарную задачу
    "columns": {
        "numeric_features": num_features,           # численные фичи
        "categorical_features": ["LatBin"],         # категориальные фичи (по желанию)
        "score_column": "score",                    # вероятность положительного класса
        "prediction_column": "pred",                # метка класса
        "date_column": "date",
        "target_column": "target",
        "sample_column": "sample",
        "id_column": "id"
    }
}

# PSI-калькулятор для M1
psi_calc = PSICalculatorExtended()

In [8]:

# ================================================
# 3) Define & Run Groups M0–M4
# ================================================
# ---------- M0 ----------
group_m0 = TestGroupBuilder(group_code="M0", group_name="M0. Data Quality")
group_m0.add_test(M01_DataSummaryTest())
group_m0.add_test(M02_SampleOverlayTest())
group_m0.add_test(M03_ObsVsPredTest())

group_m0.run_all_tests(test_params={
    "M 0.1": {"df": df, "config": config},
    "M 0.2": {"df": df, "config": config},
    "M 0.3": {"df": df, "config": config},
})


In [9]:

# ---------- M1 ----------
group_m1 = TestGroupBuilder(group_code="M1", group_name="M1. Distribution & PSI")
group_m1.add_test(M11_TrainVsGenpopPSITest(psi_calc=psi_calc))
group_m1.add_test(M12_OOSVsGenpopPSITest(psi_calc=psi_calc))
group_m1.add_test(M13_MaxDateRecencyTest())
group_m1.add_test(M14_DataQualityOverviewTest())
group_m1.add_test(M15_MissingValuesTest())

group_m1.run_all_tests(test_params={
    "M 1.1": {"df": df, "config": config},
    "M 1.2": {"df": df, "config": config},
    "M 1.3": {"df": df, "config": config},
    "M 1.4": {"df": df, "config": config},
    "M 1.5": {"df": df, "config": config},
})


M 1.2 - Processing Features: 100%|██████████| 9/9 [00:05<00:00,  1.77it/s]


In [13]:

# ---------- M2 (Gini — классификация) ----------
group_m2 = TestGroupBuilder(group_code="M2", group_name="M2. Gini Tests")
group_m2.add_test(M21_GiniTest())
group_m2.add_test(M22_GiniFactorsTest())
group_m2.add_test(M24_GiniDynamicsTest())
group_m2.add_test(M25_GiniUpliftTest())

group_m2.run_all_tests(test_params={
    "M 2.1": {"df": df, "config": config},
    "M 2.2": {"df": df, "config": config},
    "M 2.4": {"df": df, "config": config},
    "M 2.5": {"df": df, "config": config}
})


In [15]:

# ---------- M3 ----------
group_m3 = TestGroupBuilder(group_code="M3", group_name="M3. Multicollinearity")
group_m3.add_test(M33_MulticollinearityTest())

group_m3.run_all_tests(test_params={
    "M 3.3": {"df": df, "config": config}
})


Calculating VIF: 100%|██████████| 8/8 [00:00<00:00, 999.92it/s]


In [11]:

# ---------- M4 ----------
group_m4 = TestGroupBuilder(group_code="M4", group_name="M4. Model Performance")

# 4.1 — доверительные интервалы метрик (классификация)
group_m4.add_test(M41_MetricConfidenceIntervalTest(model_type='classification', n_iterations=300))
# 4.2 — достаточность размера выборки (нужна модель)
group_m4.add_test(M42_SampleSizeAdequacyTest(model_type='classification'))
# 4.3 — калибровка (только классификация)
group_m4.add_test(M43_CalibrationTest())
# 4.4 — переобучение
group_m4.add_test(M44_OverfittingTest(model_type='classification'))
# 4.5 — стабильность метрики во времени
group_m4.add_test(M45_MetricStabilityTest())
# 4.6 — качество по категориям
#group_m4.add_test(M46_CategoryQualityTest(model_type='classification'))
# 4.8 — вклад признаков (SHAP, по желанию)
if RUN_SHAP:
    group_m4.add_test(M48_FeatureContributionTest())
# 4.9 — избыточность признаков (кривая uplift)
if RUN_UPLIFT:
    group_m4.add_test(M49_UpliftTest(model_type='classification'))
# 4.10 — пропуски (динамика)
group_m4.add_test(M410_MissingValuesImpactTest(threshold=0.8))
# 4.11 — корреляция с таргетом
#group_m4.add_test(M411_TargetCorrelationTest())
# 4.12 — корреляция между фичами
#group_m4.add_test(M412_FeatureCorrelationTest(threshold=0.95))
# 4.13 — VIF-анализ
#group_m4.add_test(M413_VIFTest())
# (4.14 — TwoForestSelectionTest можно добавить при необходимости)
group_m4.add_test(M414_TwoForestSelectionTest(model_type='classification'))

# Параметры для M4:
m4_params = {
    "M 4.1": {"df": df, "config": config, "sample": "test"},
    "M 4.2": {"df": df, "config": config, "model": RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1), "sample": "train"},
    "M 4.3": {"df": df, "config": config, "sample": "test"},
    "M 4.4": {"df": df, "config": config},
    "M 4.5": {"df": df, "config": config, "sample": "test", "freq": "M"},
    #"M 4.6": {"df": df, "config": config, "category_column": "LatBin", "sample": "test"},
    "M 4.10": {"df": df, "config": config},
    #"M 4.11": {"df": df, "config": config},
    #"M 4.12": {"df": df, "config": config},
    #"M 4.13": {"df": df, "config": config},
    "M 4.14": {"df": df, "config": config}
}
if RUN_SHAP:
    m4_params["M 4.8"] = {"df": df, "config": config, "model": clf, "sample": "test"}
if RUN_UPLIFT:
    m4_params["M 4.9"] = {"df": df, "config": config, "model_class": model_class_clf}

group_m4.run_all_tests(test_params=m4_params)



 98%|===================| 197/200 [00:13<00:00]        

In [16]:

# ================================================
# 4) Build final HTML report
# ================================================
builder = ModelReportBuilder()
builder.add_group(group_m0)
builder.add_group(group_m1)
builder.add_group(group_m2)
builder.add_group(group_m3)
builder.add_group(group_m4)

html = builder.generate_html()
with open("report_california_housing_M0-M4.html", "w", encoding="utf-8") as f:
    f.write(html)

print("Готово: report_california_housing_M0-M4.html")


Готово: report_california_housing_M0-M4.html
